# install necessary packages

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install -q hazm
!pip install -q clean-text[gpl]

# Restart Session
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 35.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-n

# import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import hazm
from cleantext import clean
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections

# mount and download dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/digimag.zip -d /content/

Archive:  /content/drive/MyDrive/digimag.zip
   creating: /content/digimag/
  inflating: /content/digimag/dev.csv  
  inflating: /content/digimag/train.csv  
  inflating: /content/digimag/test.csv  


# go through dataset

In [4]:
train_file_path = '/content/digimag/train.csv'
train_data = pd.read_csv(train_file_path, encoding='utf-8', sep='\t')
train_data

,Unnamed: 0,content,label,label_id
0,0,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,علم و تکنولوژی,3
1,1,شکست Justice League در باکس آفیس پس از بازخورد...,هنر و سینما,5
2,2,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,هنر و سینما,5
3,3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,علم و تکنولوژی,3
4,4,بررسی جزء به جزء بهترین بخش Ori and the Blind ...,بازی ویدیویی,0
...,...,...,...,...
6891,6891,چند ساعت خواب برای بدن شما مفید است؟ شایع است ...,سلامت و زیبایی,2
6892,6892,فیلم مگامن ساخته می‌شود شرکت بازی‌سازی کپ‌کام ...,بازی ویدیویی,0
6893,6893,تعداد حملات بدافزاری به سیستم‌ عامل مکینتاش بر...,علم و تکنولوژی,3
6894,6894,چطور از مبتلا شدن به آنفولانزا پیشگیری کنیم؟ (...,سلامت و زیبایی,2


In [5]:
dev_file_path = '/content/digimag/dev.csv'
dev_data = pd.read_csv(dev_file_path, encoding='utf-8', sep='\t')
dev_data

,Unnamed: 0,content,label,label_id
0,0,نت‌فلیکس مجموعه‌ی سریالی کسلوانیا را می‌سازد [...,هنر و سینما,5
1,1,کارگردان The Last Guardian از بازی بعدی‌اش می‌...,بازی ویدیویی,0
2,2,کارت اعتباری اپل کارت معرفی شد در مراسم Show T...,علم و تکنولوژی,3
3,3,از کنفرانس مایکروسافت و کنسول بعدی سونی تا دیگ...,بازی ویدیویی,0
4,4,از رویداد ماه اکتبر اپل چه انتظاراتی داریم؟ در...,علم و تکنولوژی,3
...,...,...,...,...
762,762,۱۰ جزیره زیبای دنیا که حتما باید به آن‌ها سفر ...,سلامت و زیبایی,2
763,763,تریلر بسته الحاقی Iceborne برای Monster Hunter...,بازی ویدیویی,0
764,764,دیزنی از روابطش با الکترونیک آرتز می‌گوید به گ...,بازی ویدیویی,0
765,765,افتضاح لوت‌باکس‌های Battlefront ۲ سیاه‌ترین دو...,بازی ویدیویی,0


In [6]:
test_file_path = '/content/digimag/test.csv'
test_data = pd.read_csv(test_file_path, encoding='utf-8', sep='\t')
test_data

,Unnamed: 0,content,label,label_id
0,0,با این ۵ صبحانه رژیمی لاغر شوید صبحانه همیشه ب...,سلامت و زیبایی,2
1,1,نمایشگر گوشی تاشونده ال‌جی مانند بروشور باز و ...,علم و تکنولوژی,3
2,2,چرا برخی سلبریتی‌های هالیوودی از شبکه‌های اجتم...,هنر و سینما,5
3,3,گوگل استیدیا؛ آینده‌ی صنعت بازی یا محکوم به شک...,بازی ویدیویی,0
4,4,ویژگی جذاب گلکسی Z Flip به زودی به گوشی‌های ان...,علم و تکنولوژی,3
...,...,...,...,...
847,847,Death Stranding پر از میان‌پرده‌های طولانی است...,بازی ویدیویی,0
848,848,حالت نیو گیم پلاس به اسپایدرمن اضافه می‌شود چن...,بازی ویدیویی,0
849,849,بوکوفسکی؛ ملک‌الشعرای فرودستان آمریکا چارلز بو...,کتاب و ادبیات,6
850,850,چگونه مثل جف بزوس هک نشویم؛ ۵ توصیه امنیتی برا...,علم و تکنولوژی,3


remove Unnamed column

In [7]:
train_data.drop('Unnamed: 0', axis=1, inplace=True)
train_data

,content,label,label_id
0,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,علم و تکنولوژی,3
1,شکست Justice League در باکس آفیس پس از بازخورد...,هنر و سینما,5
2,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,هنر و سینما,5
3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,علم و تکنولوژی,3
4,بررسی جزء به جزء بهترین بخش Ori and the Blind ...,بازی ویدیویی,0
...,...,...,...
6891,چند ساعت خواب برای بدن شما مفید است؟ شایع است ...,سلامت و زیبایی,2
6892,فیلم مگامن ساخته می‌شود شرکت بازی‌سازی کپ‌کام ...,بازی ویدیویی,0
6893,تعداد حملات بدافزاری به سیستم‌ عامل مکینتاش بر...,علم و تکنولوژی,3
6894,چطور از مبتلا شدن به آنفولانزا پیشگیری کنیم؟ (...,سلامت و زیبایی,2


In [8]:
dev_data.drop('Unnamed: 0', axis=1, inplace=True)
dev_data

,content,label,label_id
0,نت‌فلیکس مجموعه‌ی سریالی کسلوانیا را می‌سازد [...,هنر و سینما,5
1,کارگردان The Last Guardian از بازی بعدی‌اش می‌...,بازی ویدیویی,0
2,کارت اعتباری اپل کارت معرفی شد در مراسم Show T...,علم و تکنولوژی,3
3,از کنفرانس مایکروسافت و کنسول بعدی سونی تا دیگ...,بازی ویدیویی,0
4,از رویداد ماه اکتبر اپل چه انتظاراتی داریم؟ در...,علم و تکنولوژی,3
...,...,...,...
762,۱۰ جزیره زیبای دنیا که حتما باید به آن‌ها سفر ...,سلامت و زیبایی,2
763,تریلر بسته الحاقی Iceborne برای Monster Hunter...,بازی ویدیویی,0
764,دیزنی از روابطش با الکترونیک آرتز می‌گوید به گ...,بازی ویدیویی,0
765,افتضاح لوت‌باکس‌های Battlefront ۲ سیاه‌ترین دو...,بازی ویدیویی,0


In [9]:
test_data.drop('Unnamed: 0', axis=1, inplace=True)
test_data

,content,label,label_id
0,با این ۵ صبحانه رژیمی لاغر شوید صبحانه همیشه ب...,سلامت و زیبایی,2
1,نمایشگر گوشی تاشونده ال‌جی مانند بروشور باز و ...,علم و تکنولوژی,3
2,چرا برخی سلبریتی‌های هالیوودی از شبکه‌های اجتم...,هنر و سینما,5
3,گوگل استیدیا؛ آینده‌ی صنعت بازی یا محکوم به شک...,بازی ویدیویی,0
4,ویژگی جذاب گلکسی Z Flip به زودی به گوشی‌های ان...,علم و تکنولوژی,3
...,...,...,...
847,Death Stranding پر از میان‌پرده‌های طولانی است...,بازی ویدیویی,0
848,حالت نیو گیم پلاس به اسپایدرمن اضافه می‌شود چن...,بازی ویدیویی,0
849,بوکوفسکی؛ ملک‌الشعرای فرودستان آمریکا چارلز بو...,کتاب و ادبیات,6
850,چگونه مثل جف بزوس هک نشویم؛ ۵ توصیه امنیتی برا...,علم و تکنولوژی,3


analyze train data

In [10]:
print('data information')
print('-------------------')
print(train_data.info(), '\n')

data information
-------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   6896 non-null   object
 1   label     6896 non-null   object
 2   label_id  6896 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 161.8+ KB
None 



In [11]:
print('missing values stats')
print('-------------------')
print(train_data.isnull().sum(), '\n')

missing values stats
-------------------
content     0
label       0
label_id    0
dtype: int64 



In [12]:
print('some missing values')
print('-------------------')
print(train_data[train_data['label_id'].isnull()].iloc[:5], '\n')

some missing values
-------------------
Empty DataFrame
Columns: [content, label, label_id]
Index: [] 



In [13]:
print(f"length of train data is: {len(train_data)}")
print(f"length of dev data is: {len(dev_data)}")
print(f"length of test data is: {len(test_data)}")

length of train data is: 6896
length of dev data is: 767
length of test data is: 852


sampling

In [14]:
train_data = train_data[:7000] # roughly 80% # equivalent to train_data[:6895]
dev_data = dev_data[:700] # 10%
test_data = test_data[:700] # 10%
print(f"length of train data after sampling is: {len(train_data)}")
print(f"length of dev data after sampling is: {len(dev_data)}")
print(f"length of test data after sampling is: {len(test_data)}")

length of train data after sampling is: 6896
length of dev data after sampling is: 700
length of test data after sampling is: 700


In [15]:
unique_rates = list(sorted(train_data['label_id'].unique()))
print(f'num of unique labels: {len(unique_rates)} \n label_ids: {unique_rates}')

num of unique labels: 7 
 label_ids: [0, 1, 2, 3, 4, 5, 6]


In [16]:
fig = go.Figure()

groupby_rate = train_data.groupby('label_id')['label_id'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label ids within contents',
    xaxis_title_text='Label ID',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [17]:
# Function to merge the least common labels into an 'other' class
def merge_least_common_labels(df, label_column, least_common_labels, new_label='سایر'):
    # Replace the least common labels with the new label
    df[label_column] = df[label_column].apply(lambda x: new_label if x in least_common_labels else x)
    return df

# Function to balance the dataset
def balance_dataset(df, label_column, min_count):
    balanced_df = df.groupby(label_column).apply(lambda x: x.sample(min_count)).reset_index(drop=True)
    return balanced_df

def balance_data(data):
  # Count the number of instances for each label
  label_counts = data['label'].value_counts()
  print(label_counts)

  # Identify the three least common labels
  least_common_labels = label_counts.nsmallest(3).index
  print(f"Least common labels: {least_common_labels}")

  # Merge the least common labels
  merged_data = merge_least_common_labels(data, 'label', least_common_labels)

  # Recalculate the label counts after merging
  label_counts_after_merge = merged_data['label'].value_counts()
  print(label_counts_after_merge)

  # Reset label_ids so that there is no gap between them
  label_mapping = {label: idx for idx, label in enumerate(sorted(label_counts_after_merge.index))}
  print(f"New label mapping: {label_mapping}")

  merged_data['label_id'] = merged_data['label'].map(label_mapping)

  # Verify the new label_ids
  print(merged_data[['label', 'label_id']].drop_duplicates())

  # Find the minimum count after merging
  min_count_after_merge = label_counts_after_merge.min()
  print(f"Minimum count after merging: {min_count_after_merge}")

  # Balance the dataset after merging
  balanced_data = balance_dataset(merged_data, 'label', min_count_after_merge)

  return balanced_data

In [18]:
train_data = balance_data(train_data)

label
علم و تکنولوژی    2245
بازی ویدیویی      1593
هنر و سینما       1350
سلامت و زیبایی    1304
کتاب و ادبیات      206
راهنمای خرید       101
عمومی               97
Name: count, dtype: int64
Least common labels: Index(['عمومی', 'راهنمای خرید', 'کتاب و ادبیات'], dtype='object', name='label')
label
علم و تکنولوژی    2245
بازی ویدیویی      1593
هنر و سینما       1350
سلامت و زیبایی    1304
سایر               404
Name: count, dtype: int64
New label mapping: {'بازی ویدیویی': 0, 'سایر': 1, 'سلامت و زیبایی': 2, 'علم و تکنولوژی': 3, 'هنر و سینما': 4}
            label  label_id
0  علم و تکنولوژی         3
1     هنر و سینما         4
4    بازی ویدیویی         0
6            سایر         1
8  سلامت و زیبایی         2
Minimum count after merging: 404


In [19]:
dev_data = balance_data(dev_data)

label
علم و تکنولوژی    223
بازی ویدیویی      157
سلامت و زیبایی    141
هنر و سینما       136
کتاب و ادبیات      23
عمومی              11
راهنمای خرید        9
Name: count, dtype: int64
Least common labels: Index(['راهنمای خرید', 'عمومی', 'کتاب و ادبیات'], dtype='object', name='label')
label
علم و تکنولوژی    223
بازی ویدیویی      157
سلامت و زیبایی    141
هنر و سینما       136
سایر               43
Name: count, dtype: int64
New label mapping: {'بازی ویدیویی': 0, 'سایر': 1, 'سلامت و زیبایی': 2, 'علم و تکنولوژی': 3, 'هنر و سینما': 4}
             label  label_id
0      هنر و سینما         4
1     بازی ویدیویی         0
2   علم و تکنولوژی         3
6   سلامت و زیبایی         2
16            سایر         1
Minimum count after merging: 43


<ipython-input-17-5be295e30afc>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-17-5be295e30afc>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
test_data = balance_data(test_data)

label
علم و تکنولوژی    223
بازی ویدیویی      160
هنر و سینما       143
سلامت و زیبایی    135
کتاب و ادبیات      20
عمومی              10
راهنمای خرید        9
Name: count, dtype: int64
Least common labels: Index(['راهنمای خرید', 'عمومی', 'کتاب و ادبیات'], dtype='object', name='label')
label
علم و تکنولوژی    223
بازی ویدیویی      160
هنر و سینما       143
سلامت و زیبایی    135
سایر               39
Name: count, dtype: int64
New label mapping: {'بازی ویدیویی': 0, 'سایر': 1, 'سلامت و زیبایی': 2, 'علم و تکنولوژی': 3, 'هنر و سینما': 4}
             label  label_id
0   سلامت و زیبایی         2
1   علم و تکنولوژی         3
2      هنر و سینما         4
3     بازی ویدیویی         0
41            سایر         1
Minimum count after merging: 39


<ipython-input-17-5be295e30afc>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-17-5be295e30afc>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
# Verify the distribution
print(train_data['label'].value_counts())
print(dev_data['label'].value_counts())
print(test_data['label'].value_counts())

label
بازی ویدیویی      404
سایر              404
سلامت و زیبایی    404
علم و تکنولوژی    404
هنر و سینما       404
Name: count, dtype: int64
label
بازی ویدیویی      43
سایر              43
سلامت و زیبایی    43
علم و تکنولوژی    43
هنر و سینما       43
Name: count, dtype: int64
label
بازی ویدیویی      39
سایر              39
سلامت و زیبایی    39
علم و تکنولوژی    39
هنر و سینما       39
Name: count, dtype: int64


In [22]:
fig = go.Figure()

groupby_label = train_data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within contents [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

# Preprocessing data

In [25]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)

    # normalizing # we comment this section because it takes too long time :)
    # normalizer = hazm.Normalizer()
    # text = normalizer.normalize(text)

    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

def preprocess_dataset(data):
  tqdm.pandas()
  data['content'] = data['content'].progress_apply(cleaning)

  return data

In [26]:
train_data = preprocess_dataset(train_data)
train_data

  0%|          | 0/2020 [00:00<?, ?it/s]

,content,label,label_id
0,تریلر محیط‌های blair witch سازندگان بازی blair...,بازی ویدیویی,0
1,طراح دیابلو راه‌حل مشکلات anthem را می‌داند به...,بازی ویدیویی,0
2,بلیزارد مشغول ساخت یک بازی موبایل وارکرفت است ...,بازی ویدیویی,0
3,تریلر رویداد اونجرز در fortnite مثل سال گذشته،...,بازی ویدیویی,0
4,شعبه‌ی لس‌آنجلس dice به وینسنت زمپلا سپرده شد ...,بازی ویدیویی,0
...,...,...,...
2015,اصغر فرهادی؛ تنها کارگردان برنده‌ی اسکار ایران...,هنر و سینما,4
2016,[ویدیو] پرفروش‌ترین فیلم‌ها در سال ۲۰۱۶ سال ۲۰...,هنر و سینما,4
2017,رادیوپل: مار و مارگیر توی روزهای عید که بهترین...,هنر و سینما,4
2018,مارک همیل احتمالا نه تنها در قسمت هفتم، بلکه د...,هنر و سینما,4


In [27]:
val_data = preprocess_dataset(dev_data)
val_data

  0%|          | 0/215 [00:00<?, ?it/s]

,content,label,label_id
0,لوگوی ایکس‌باکس اسکورپیو لو رفت مایکروسافت امر...,بازی ویدیویی,0
1,تاریخ عرضه‌ی detroit: become human مشخص شد سون...,بازی ویدیویی,0
2,از کنفرانس مایکروسافت و کنسول بعدی سونی تا دیگ...,بازی ویدیویی,0
3,بتسدا: قطعا ولفنشتاین ۳ را می‌سازیم این طور که...,بازی ویدیویی,0
4,کارگردان anthem می‌گوید که بازی احساسی‌ترین اک...,بازی ویدیویی,0
...,...,...,...
210,نظر منتقدان درباره پرفروش‌ترین فیلم‌های ۲۰۱۸ ت...,هنر و سینما,4
211,۵۱ فیلمی که در سال ۲۰۱۹ نباید از دست بدهید پیش...,هنر و سینما,4
212,از جوکر تا انگل؛ نگاهی به آثار شاخص جشنواره بی...,هنر و سینما,4
213,فصل ۷ سریال game of thrones تاخیر خورد قسمت آخ...,هنر و سینما,4


In [28]:
test_data = preprocess_dataset(test_data)
test_data

  0%|          | 0/195 [00:00<?, ?it/s]

,content,label,label_id
0,از بازسازی رزیدنت اویل ۳ چه انتظاراتی داریم؟ پ...,بازی ویدیویی,0
1,دل‌سوزی‌های مایکروسافتی و آینده‌ی کنسول نینتند...,بازی ویدیویی,0
2,کوجیما از ساختار death stranding می‌گوید؛ بازی...,بازی ویدیویی,0
3,کارگردان گاد آو وار: تمام بازی‌ها تا لحظه‌ی آخ...,بازی ویدیویی,0
4,آنچه شما گفتید: بهترین بازی ۲۰۱۷ در قالب مطلب ...,بازی ویدیویی,0
...,...,...,...
190,نقد فیلم جوکر؛ تکثیر قانونی جوکرها در جامعه در...,هنر و سینما,4
191,پیش‌بینی برندگان اسکار ۲۰۱۹ در بخش‌های اصلی پی...,هنر و سینما,4
192,طرفداران کاوه آفاق در کنسرتش غافلگیر شدند کاوه...,هنر و سینما,4
193,رادیوپل: ویتنام جهنم آمریکایی‌ها در پادکست شما...,هنر و سینما,4


In [29]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

dataset = DatasetDict(
    train=train_dataset,
    val=val_dataset,
    test=test_dataset
)
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'label', 'label_id'],
        num_rows: 2020
    })
    val: Dataset({
        features: ['content', 'label', 'label_id'],
        num_rows: 215
    })
    test: Dataset({
        features: ['content', 'label', 'label_id'],
        num_rows: 195
    })
})

In [30]:
dataset['train'][10]

{'content': 'مشکلات mass effect جدید از کجا ریشه گرفته\u200cاند؟ بازخورد نه چندان خوب جدیدترین قسمت مجموعه بازی\u200cهای مس افکت (mass effect) ظاهرا به مشکلات زیادی که در ساخت آن وجود داشته، بازمی\u200cگردد\u200c مقاله\u200cای بلند و بالا از مراحل ساخت و توسعه\u200cی مس افکت: آندرومدا (mass effect: andromeda) در وب\u200cسایت کوتاکو منتشر شده است که نگاهی به پشت صحنه\u200cی ساخت این بازی می\u200cاندازد\u200c این مقاله از این موضوع می\u200cگوید که چرا از پنج سالی که صرف ساخت مس افکت جدید شده، فقط یک و نیم سال را واقعا مشغول ساخت بازی بوده\u200cاند\u200cشعبه\u200cی مونترال استودیوی بازی\u200cسازی بایوویر در سال ۲۰۱۲ وظیفه\u200cی ساخت قسمت چهارم مس افکت را برعهده می\u200cگیرد؛ آن هم در حالی که شعبه\u200cی ادمونتون بایوویر سه\u200cگانه\u200cی اصلی مس افکت را ساخته بود\u200c تیم سازنده\u200cی بازی، ایده\u200cهایی بزرگ را برای مس افکت: آندرومدا در سر می\u200cپروراند؛ ایده\u200cهایی مثل سیاره\u200cهای بی\u200cنهایتی که خودشان ساخته می\u200cشدند، انیمیشن صورت بسیار واقعی و استفاده\u200cی کامل و

In [31]:
dataset['test'][5]

{'content': 'ریزر از یک ماوس و هدست overwatch رونمایی کرد یکی دو روز پیش شاهد رونمایی یک شخصیت و نقشه\u200cی جدید برای اورواچ (overwatch) بودیم. شرکت ریزر هم در طول رویداد بلیزکان ۲۰۱۷ از محصولات جدیدی برای overwatch رونمایی کرد. این محصولات همگی با الهام از شخصیت d-va طراحی شده\u200cاند. محصولات ریزر برای شخصیت d-va شامل یک ماوس، یک هدست و یک ماوس پد می\u200cشود. هدست razer meka رنگ سیاه، صورتی، سبز و زرد را با هم ادغام کرده است و ظاهری شبیه به هدست خود شخصیت d-va دارد. در دنیای اورواچ، شخصیت d-va یک گیمر ۱۹ ساله کره\u200cای است که به صورت حرفه\u200cای در مسابقات استارکرفت بازی می\u200cکرده. شرکت ریزر حرفی از زمان عرضه\u200cی هدست meka نزده و حتی اطلاعات بیشتری از آن ارایه نداده است. فقط می\u200cدانیم که این هدست\u200cها شبیه به هدست d-va ساخته شده\u200cاند. به غیر از این هدست، ریزر از یک ماوس و ماوس پد که به ترتیب razer abyssus elite و d-va razer goliathus نام گرفته\u200cاند، معرفی کرد. این دو ماوس و ماوس پد را می\u200cتوان همین حالا خریداری کرد. ماوس d-va abyssus elite یک سنسور اپتی

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ViraIntelligentDataMining/AriaBERT")
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



RobertaTokenizerFast(name_or_path='ViraIntelligentDataMining/AriaBERT', vocab_size=60000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

# Load Tokenizer and Tokenize dataset

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ViraIntelligentDataMining/AriaBERT")
tokenizer

RobertaTokenizerFast(name_or_path='ViraIntelligentDataMining/AriaBERT', vocab_size=60000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

tokenize dataset

In [34]:
from torch.utils.data import DataLoader

# Define a function to tokenize the dataset
def tokenize(batch):
    return tokenizer(batch['content'], padding=True, truncation=True, max_length=512)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=16)
tokenized_dataset

Map:   0%|          | 0/2020 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'label', 'label_id', 'input_ids', 'attention_mask'],
        num_rows: 2020
    })
    val: Dataset({
        features: ['content', 'label', 'label_id', 'input_ids', 'attention_mask'],
        num_rows: 215
    })
    test: Dataset({
        features: ['content', 'label', 'label_id', 'input_ids', 'attention_mask'],
        num_rows: 195
    })
})

In [35]:
tokenized_dataset = tokenized_dataset.remove_columns(["label"])
tokenized_dataset = tokenized_dataset.remove_columns(["content"])
tokenized_dataset = tokenized_dataset.rename_column("label_id", "labels")

In [36]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2020
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 215
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 195
    })
})

In [37]:
unique_labels = set(tokenized_dataset['train']['labels'])
num_unique_labels = len(unique_labels)
print(f"unique_labels: {unique_labels}")
print(f"num_unique_labels: {num_unique_labels}")

unique_labels: {0, 1, 2, 3, 4}
num_unique_labels: 5


In [38]:
tokenized_dataset.set_format("torch") # Set the format of the dataset to PyTorch tensors

# Create dataloaders
train_dataloader = DataLoader(tokenized_dataset['train'], batch_size=16)
eval_dataloader = DataLoader(tokenized_dataset['val'], batch_size=16)
test_dataloader = DataLoader(tokenized_dataset['test'], batch_size=16)

# Load and Train Model

In [ ]:
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification
from collections import defaultdict, Counter
import json
import torch

model = AutoModelForSequenceClassification.from_pretrained('ViraIntelligentDataMining/AriaBERT', num_labels=num_unique_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ViraIntelligentDataMining/AriaBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
input_str = "سلام من فرزان هستم."
model_inputs = tokenizer(input_str, return_tensors="pt")

model_outputs = model(**model_inputs)

print(model_inputs)
print("----------------------")
print(model_outputs)
print("----------------------")
print(f"Distribution over labels: {torch.softmax(model_outputs.logits, dim=1)}")

{'input_ids': tensor([[    0,  1901,   371, 31991,  2545,    18,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
----------------------
SequenceClassifierOutput(loss=None, logits=tensor([[-0.1101,  0.3163, -0.3453, -0.3998, -0.1555]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
----------------------
Distribution over labels: tensor([[0.1990, 0.3048, 0.1573, 0.1489, 0.1901]], grad_fn=<SoftmaxBackward0>)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Initialize data collator
data_collator = DataCollatorWithPadding(tokenizer)

def compute_metrics(eval_pred):
    """Called at the end of validation. Calculates accuracy, precision, recall, and F1 score."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

arguments = TrainingArguments(
    output_dir="digimag_classifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=128,
    warmup_steps=500,
    weight_decay=0.01
)

In [ ]:
trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Mon May 20 17:21:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |   2283MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # Set CUDA_LAUNCH_BLOCKING to get more precise error messages

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['labels', 'input_ids', 'attention_mask'])


In [ ]:
# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss()

model.to(device)
batch = next(iter(train_dataloader))

# Move each tensor in the batch to the GPU
batch = {k: v.to(device) for k, v in batch.items()}

outputs = model(**batch)
loss = loss_fn(outputs.logits, batch['labels'])
loss.backward()

In [ ]:
print("Input shape:", batch['input_ids'].shape)
print("Output shape:", outputs.logits.shape)
print("Labels shape:", batch['labels'].shape)

Input shape: torch.Size([16, 512])
Output shape: torch.Size([16, 5])
Labels shape: torch.Size([16])


test data loaders

In [ ]:
for batch in train_dataloader:
    labels = batch['labels']
    print(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def check_label_range(dataloader, num_classes):
    for batch in dataloader:
        labels = batch['labels']
        if labels.min() < 0 or labels.max() >= num_classes:
            print(labels)
            raise ValueError(f"Labels out of range: {labels.min()} - {labels.max()}")
    print("All labels are within the correct range.")

num_classes = num_unique_labels  # Set this to the number of classes in your classification task
check_label_range(train_dataloader, num_classes)
check_label_range(eval_dataloader, num_classes)

All labels are within the correct range.
All labels are within the correct range.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.267232,0.697674,0.728748,0.697674,0.699913
2,No log,0.503103,0.855814,0.866661,0.855814,0.851766
3,No log,0.358809,0.883721,0.890189,0.883721,0.880197


TrainOutput(global_step=381, training_loss=0.9040140467365896, metrics={'train_runtime': 685.5786, 'train_samples_per_second': 8.839, 'train_steps_per_second': 0.556, 'total_flos': 1594495943331840.0, 'train_loss': 0.9040140467365896, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
results

{'eval_loss': 0.35880908370018005,
 'eval_accuracy': 0.8837209302325582,
 'eval_precision': 0.8901893239565328,
 'eval_recall': 0.8837209302325582,
 'eval_f1': 0.8801971382867022,
 'eval_runtime': 7.9311,
 'eval_samples_per_second': 27.108,
 'eval_steps_per_second': 1.765,
 'epoch': 3.0}

push to hub

In [ ]:
from google.colab import userdata

# push to huggin face
USERNAME=userdata.get('HUGGINGFACE_USERNAME')
TOKEN=userdata.get('HUGGINGFACE_WRITE_ACCESS_TOKEN')
model.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_3_lr_2e_5_unfreezed", token=TOKEN, commit_message="Upload AriaBERT Model finetuned on digimag dataset")
tokenizer.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_3_lr_2e_5_unfreezed", token=TOKEN, commit_message="Upload AriaBERT Tokenizer finetuned on digimag dataset")

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/farzanrahmani/AriaBERT_finetuned_digimag_Epoch_3_lr_2e_5_unfreezed/commit/4178b8b142f5189aa38131c4574ea9fb753e59f5', commit_message='Upload AriaBERT Tokenizer finetuned on digimag dataset', commit_description='', oid='4178b8b142f5189aa38131c4574ea9fb753e59f5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Mon May 20 17:22:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              37W /  70W |   4787MiB / 15360MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# test model with different learning rate

In [39]:
!mkdir checkpoints
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification
from collections import defaultdict, Counter
import json
import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
data_collator = DataCollatorWithPadding(tokenizer)
def compute_metrics(eval_pred):
    """Called at the end of validation. Calculates accuracy, precision, recall, and F1 score."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # Set CUDA_LAUNCH_BLOCKING to get more precise error messages

In [40]:
# Load another model and freeze its base model parameters
tokenizer = AutoTokenizer.from_pretrained("ViraIntelligentDataMining/AriaBERT")
different_lr_model = AutoModelForSequenceClassification.from_pretrained('ViraIntelligentDataMining/AriaBERT', num_labels=num_unique_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/529M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ViraIntelligentDataMining/AriaBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
different_lr_arguments = TrainingArguments(
    output_dir="digimag_classifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",  # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=3e-4,
    load_best_model_at_end=True,
    seed=224,
    warmup_steps=500,
    weight_decay=0.01
)

different_lr_trainer = Trainer(
    model=different_lr_model,
    args=different_lr_arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [42]:
# Train the frozen model
different_lr_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.434350,0.860465,0.863530,0.860465,0.859260
2,No log,0.593714,0.837209,0.852873,0.837209,0.833444
3,No log,0.599995,0.846512,0.859357,0.846512,0.849342


TrainOutput(global_step=381, training_loss=0.5615600425740239, metrics={'train_runtime': 661.5788, 'train_samples_per_second': 9.16, 'train_steps_per_second': 0.576, 'total_flos': 1594495943331840.0, 'train_loss': 0.5615600425740239, 'epoch': 3.0})

In [43]:
# Evaluate the frozen model
different_lr_results = different_lr_trainer.evaluate()
different_lr_results

{'eval_loss': 0.43434980511665344,
 'eval_accuracy': 0.8604651162790697,
 'eval_precision': 0.863529508574438,
 'eval_recall': 0.8604651162790697,
 'eval_f1': 0.8592603521615074,
 'eval_runtime': 7.0044,
 'eval_samples_per_second': 30.695,
 'eval_steps_per_second': 1.999,
 'epoch': 3.0}

In [45]:
from google.colab import userdata

# push to huggin face
USERNAME=userdata.get('HUGGINGFACE_USERNAME')
TOKEN=userdata.get('HUGGINGFACE_WRITE_ACCESS_TOKEN')
different_lr_model.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_3_lr_3e_4_unfreezed", token=TOKEN, commit_message="Upload AriaBERT Model finetuned on digimag dataset")
tokenizer.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_3_lr_3e_4_unfreezed", token=TOKEN, commit_message="Upload AriaBERT Tokenizer finetuned on digimag dataset")

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/farzanrahmani/AriaBERT_finetuned_digimag_Epoch_3_lr_3e_4_unfreezed/commit/3af4ff7dece241a11586968b9f4c0a0345dc9467', commit_message='Upload AriaBERT Tokenizer finetuned on digimag dataset', commit_description='', oid='3af4ff7dece241a11586968b9f4c0a0345dc9467', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Mon May 20 17:51:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              35W /  70W |   1969MiB / 15360MiB |     27%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

!! as you can see lr=2e-5 is better than lr=3e-4

-------------------------
```
lr=2e-5
{'eval_loss': 0.35880908370018005,
 'eval_accuracy': 0.8837209302325582,
 'eval_precision': 0.8901893239565328,
 'eval_recall': 0.8837209302325582,
 'eval_f1': 0.8801971382867022,
 'eval_runtime': 7.9311,
 'eval_samples_per_second': 27.108,
 'eval_steps_per_second': 1.765,
 'epoch': 3.0}
```


-------------------------
```
lr=3e-4
{'eval_loss': 0.43434980511665344,
 'eval_accuracy': 0.8604651162790697,
 'eval_precision': 0.863529508574438,
 'eval_recall': 0.8604651162790697,
 'eval_f1': 0.8592603521615074,
 'eval_runtime': 7.0044,
 'eval_samples_per_second': 30.695,
 'eval_steps_per_second': 1.999,
 'epoch': 3.0}
```

# test Model with Freezed layers

In [ ]:
!mkdir checkpoints
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification
from collections import defaultdict, Counter
import json
import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
data_collator = DataCollatorWithPadding(tokenizer)
def compute_metrics(eval_pred):
    """Called at the end of validation. Calculates accuracy, precision, recall, and F1 score."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # Set CUDA_LAUNCH_BLOCKING to get more precise error messages

In [48]:
# Load another model and freeze its base model parameters
tokenizer = AutoTokenizer.from_pretrained("ViraIntelligentDataMining/AriaBERT")
freezed_model = AutoModelForSequenceClassification.from_pretrained('ViraIntelligentDataMining/AriaBERT', num_labels=num_unique_labels)

# Freeze the base model parameters
for param in freezed_model.base_model.parameters():
    param.requires_grad = False
    param.trainable = False

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ViraIntelligentDataMining/AriaBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
freezed_arguments = TrainingArguments(
    output_dir="digimag_classifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",  # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224,
    warmup_steps=500,
    weight_decay=0.01
)

freezed_trainer = Trainer(
    model=freezed_model,
    args=freezed_arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [50]:
# Train the frozen model
freezed_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.560253,0.376744,0.415126,0.376744,0.321168
2,No log,1.408770,0.679070,0.671213,0.679070,0.653922
3,No log,1.188237,0.748837,0.747490,0.748837,0.737450


TrainOutput(global_step=381, training_loss=1.481706113640092, metrics={'train_runtime': 266.2631, 'train_samples_per_second': 22.759, 'train_steps_per_second': 1.431, 'total_flos': 1594495943331840.0, 'train_loss': 1.481706113640092, 'epoch': 3.0})

In [51]:
# Evaluate the frozen model
freezed_results = freezed_trainer.evaluate()
freezed_results

{'eval_loss': 1.1882374286651611,
 'eval_accuracy': 0.7488372093023256,
 'eval_precision': 0.7474903137899653,
 'eval_recall': 0.7488372093023256,
 'eval_f1': 0.7374499206797344,
 'eval_runtime': 8.5478,
 'eval_samples_per_second': 25.153,
 'eval_steps_per_second': 1.638,
 'epoch': 3.0}

In [54]:
from google.colab import userdata

# push to huggin face
USERNAME=userdata.get('HUGGINGFACE_USERNAME')
TOKEN=userdata.get('HUGGINGFACE_WRITE_ACCESS_TOKEN')
freezed_model.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_3_lr_2e_5_freezed", token=TOKEN, commit_message="Upload AriaBERT Model finetuned on digimag dataset")
tokenizer.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_3_lr_2e_5_freezed", token=TOKEN, commit_message="Upload AriaBERT Tokenizer finetuned on digimag dataset")

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/farzanrahmani/AriaBERT_finetuned_digimag_Epoch_3_lr_2e_5_freezed/commit/5cd07ec22f6ebb0f9963e337b5e79f56c41b9db4', commit_message='Upload AriaBERT Tokenizer finetuned on digimag dataset', commit_description='', oid='5cd07ec22f6ebb0f9963e337b5e79f56c41b9db4', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Mon May 20 18:02:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              34W /  70W |   2289MiB / 15360MiB |      9%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# test Model with Freezed layers with more epochs (6 epochs)

In [56]:
# Load another model and freeze its base model parameters
tokenizer = AutoTokenizer.from_pretrained("ViraIntelligentDataMining/AriaBERT")
freezed_model = AutoModelForSequenceClassification.from_pretrained('ViraIntelligentDataMining/AriaBERT', num_labels=num_unique_labels)

# Freeze the base model parameters
for param in freezed_model.base_model.parameters():
    param.requires_grad = False
    param.trainable = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ViraIntelligentDataMining/AriaBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
freezed_arguments = TrainingArguments(
    output_dir="digimag_classifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    evaluation_strategy="epoch",  # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224,
    warmup_steps=500,
    weight_decay=0.01
)

freezed_trainer = Trainer(
    model=freezed_model,
    args=freezed_arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [58]:
# Train the frozen model
freezed_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.560766,0.413953,0.461972,0.413953,0.392451
2,No log,1.406862,0.683721,0.697291,0.683721,0.652836
3,No log,1.180172,0.804651,0.817108,0.804651,0.796492
4,1.380400,0.946222,0.800000,0.806381,0.800000,0.793496
5,1.380400,0.811388,0.804651,0.804355,0.804651,0.801050
6,1.380400,0.774177,0.809302,0.808655,0.809302,0.805961


TrainOutput(global_step=762, training_loss=1.196133911453207, metrics={'train_runtime': 492.6418, 'train_samples_per_second': 24.602, 'train_steps_per_second': 1.547, 'total_flos': 3188991886663680.0, 'train_loss': 1.196133911453207, 'epoch': 6.0})

In [59]:
# Evaluate the frozen model
freezed_results = freezed_trainer.evaluate()
freezed_results

{'eval_loss': 0.7741773128509521,
 'eval_accuracy': 0.8093023255813954,
 'eval_precision': 0.8086545682102628,
 'eval_recall': 0.8093023255813954,
 'eval_f1': 0.8059611606599557,
 'eval_runtime': 6.9604,
 'eval_samples_per_second': 30.889,
 'eval_steps_per_second': 2.011,
 'epoch': 6.0}

In [60]:
from google.colab import userdata

# push to huggin face
USERNAME=userdata.get('HUGGINGFACE_USERNAME')
TOKEN=userdata.get('HUGGINGFACE_WRITE_ACCESS_TOKEN')
freezed_model.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_6_lr_2e_5_freezed", token=TOKEN, commit_message="Upload AriaBERT Model finetuned on digimag dataset")
tokenizer.push_to_hub(f"{USERNAME}/AriaBERT_finetuned_digimag_Epoch_6_lr_2e_5_freezed", token=TOKEN, commit_message="Upload AriaBERT Tokenizer finetuned on digimag dataset")

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/farzanrahmani/AriaBERT_finetuned_digimag_Epoch_6_lr_2e_5_freezed/commit/21691796500ec0f8558b414cc37b5a9ab81baeaf', commit_message='Upload AriaBERT Tokenizer finetuned on digimag dataset', commit_description='', oid='21691796500ec0f8558b414cc37b5a9ab81baeaf', pr_url=None, pr_revision=None, pr_num=None)

In [61]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Mon May 20 18:11:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0              32W /  70W |   2545MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

as you can see 6 Epochs have better result than 3 Epoch
------------------
6 Epochs
```
{'eval_loss': 0.7741773128509521,
 'eval_accuracy': 0.8093023255813954,
 'eval_precision': 0.8086545682102628,
 'eval_recall': 0.8093023255813954,
 'eval_f1': 0.8059611606599557,
 'eval_runtime': 6.9604,
 'eval_samples_per_second': 30.889,
 'eval_steps_per_second': 2.011,
 'epoch': 6.0}
```
------------------
3 Epochs
```
{'eval_loss': 1.1882374286651611,
 'eval_accuracy': 0.7488372093023256,
 'eval_precision': 0.7474903137899653,
 'eval_recall': 0.7488372093023256,
 'eval_f1': 0.7374499206797344,
 'eval_runtime': 8.5478,
 'eval_samples_per_second': 25.153,
 'eval_steps_per_second': 1.638,
 'epoch': 3.0}
```

# Conclusions
1. best result was for unfreezed model with lr=2e-5.
```
{'eval_loss': 0.35880908370018005,
 'eval_accuracy': 0.8837209302325582,
 'eval_precision': 0.8901893239565328,
 'eval_recall': 0.8837209302325582,
 'eval_f1': 0.8801971382867022,
 'eval_runtime': 7.9311,
 'eval_samples_per_second': 27.108,
 'eval_steps_per_second': 1.765,
 'epoch': 3.0}
 ```
2. changing lr from 2e-5 to 3e-3 make the accuracy worse.
3. unfreezed model has better accuracy than freezed model.
4. adding more epochs (3 to 6) for the freezed model increased accuracy. So if we have time increasing epochs with saving the best model and eraly stopping is a good strategy